<a href="https://colab.research.google.com/github/sridhartroy/AIML/blob/main/Mllib_Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark==3.5.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.4/317.4 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.6-py2.py3-none-any.whl size=317895798 sha256=6fbd4b84d2e0f3b493e50549cb54e91eb5c3e2ab1ed497fd82e02ded255e5539
  Stored in directory: /root/.cache/pip/wheels/64/62/f3/ec15656ea4ada0523cae62a1827fe7beb55d3c8c87174aad4a
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1


In [2]:
from pyspark.sql import SparkSession
# Initialize Spark session
spark = SparkSession.builder \
    .appName("MLLib-Lab1") \
    .getOrCreate()

sc = spark.sparkContext

In [3]:
!wget https://an-utd-course.s3-us-west-1.amazonaws.com/car-milage.csv

--2025-10-18 22:01:39--  https://an-utd-course.s3-us-west-1.amazonaws.com/car-milage.csv
Resolving an-utd-course.s3-us-west-1.amazonaws.com (an-utd-course.s3-us-west-1.amazonaws.com)... 16.15.0.235, 52.219.120.169, 52.219.192.10, ...
Connecting to an-utd-course.s3-us-west-1.amazonaws.com (an-utd-course.s3-us-west-1.amazonaws.com)|16.15.0.235|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1569 (1.5K) [text/csv]
Saving to: ‘car-milage.csv’

car-milage.csv      100%[===================>]   1.53K  --.-KB/s    in 0s      

2025-10-18 22:01:39 (96.3 MB/s) - ‘car-milage.csv’ saved [1569/1569]



In [5]:
df = spark.read.option("header","true").option("inferSchema","true").csv("car-milage.csv")


In [31]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import FeatureHasher
from pyspark.ml.classification import LogisticRegression

In [8]:
df.show(5)

+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+
|  mpg|displacement| hp|torque|CRatio|RARatio|CarbBarrells|NoOfSpeed|length|width|weight|automatic|
+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+
| 18.9|       350.0|165|   260|   8.0|   2.56|           4|        3| 200.3| 69.9|  3910|        1|
| 17.0|       350.0|170|   275|   8.5|   2.56|           4|        3| 199.6| 72.9|  3860|        1|
| 20.0|       250.0|105|   185|  8.25|   2.73|           1|        3| 196.7| 72.2|  3510|        1|
|18.25|       351.0|143|   255|   8.0|    3.0|           2|        3| 199.9| 74.0|  3890|        1|
|20.07|       225.0| 95|   170|   8.4|   2.76|           1|        3| 194.1| 71.8|  3365|        0|
+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+
only showing top 5 rows



In [9]:
hasher =  FeatureHasher().setInputCols(["mpg","displacement","hp","torque","CRatio","RARatio","CarbBarrells","NoOfSpeed","length","width","weight"]).setOutputCol("features")

In [16]:
df_transformed = hasher.transform(df)
type(df_transformed)
print(df.show(5))
df_transformed.show(5, truncate=False)

+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+
|  mpg|displacement| hp|torque|CRatio|RARatio|CarbBarrells|NoOfSpeed|length|width|weight|automatic|
+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+
| 18.9|       350.0|165|   260|   8.0|   2.56|           4|        3| 200.3| 69.9|  3910|        1|
| 17.0|       350.0|170|   275|   8.5|   2.56|           4|        3| 199.6| 72.9|  3860|        1|
| 20.0|       250.0|105|   185|  8.25|   2.73|           1|        3| 196.7| 72.2|  3510|        1|
|18.25|       351.0|143|   255|   8.0|    3.0|           2|        3| 199.9| 74.0|  3890|        1|
|20.07|       225.0| 95|   170|   8.4|   2.76|           1|        3| 194.1| 71.8|  3365|        0|
+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+
only showing top 5 rows

None
+-----+------------+---+------+------+-------+------------+---------+-

In [18]:
df_transformed.head()

Row(mpg=18.9, displacement=350.0, hp=165, torque=260, CRatio=8.0, RARatio=2.56, CarbBarrells=4, NoOfSpeed=3, length=200.3, width=69.9, weight=3910, automatic=1, features=SparseVector(262144, {3416: 3.0, 3594: 8.0, 30644: 69.9, 49914: 4.0, 68404: 3910.0, 82514: 2.56, 148293: 350.0, 192774: 200.3, 221527: 165.0, 241275: 18.9, 245500: 260.0}))

In [27]:
hasher.explainParam(param="inputCols")

"inputCols: input column names. (current: ['mpg', 'displacement', 'hp', 'torque', 'CRatio', 'RARatio', 'CarbBarrells', 'NoOfSpeed', 'length', 'width', 'weight'])"

In [28]:
train, test = df_transformed.randomSplit([0.7, 0.3], seed = 2018)

In [30]:
print("DF count: " + str(df_transformed.count()))
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

DF count: 32
Training Dataset Count: 24
Test Dataset Count: 8


In [32]:
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, featuresCol = "features", labelCol = "automatic")

In [38]:
model = lr.fit(train)
type(model)

pyspark.ml.classification.LogisticRegressionModel

In [35]:
result = model.transform(test)
type(result)

pyspark.sql.dataframe.DataFrame

In [37]:
result.show(5, truncate=False)

+----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+-------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------+----------------------------------------+----------+
|mpg |displacement|hp |torque|CRatio|RARatio|CarbBarrells|NoOfSpeed|length|width|weight|automatic|features                                                                                                                                   |rawPrediction                           |probability                             |prediction|
+----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+-------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------+----------------------------------------+----------+
|11.

In [39]:
result.select("automatic","prediction").show()
# let's evaluate accuracy
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol = "prediction", labelCol = "automatic", metricName = "accuracy")
accuracy = evaluator.evaluate(result)
print(accuracy)
# try other metricnames

+---------+----------+
|automatic|prediction|
+---------+----------+
|        1|       1.0|
|        1|       1.0|
|        1|       1.0|
|        1|       1.0|
|        1|       1.0|
|        0|       1.0|
|        0|       1.0|
|        0|       1.0|
+---------+----------+

0.625


In [40]:
!wget https://an-utd-python.s3.us-west-1.amazonaws.com/yelp_labelled.txt

--2025-10-18 23:08:11--  https://an-utd-python.s3.us-west-1.amazonaws.com/yelp_labelled.txt
Resolving an-utd-python.s3.us-west-1.amazonaws.com (an-utd-python.s3.us-west-1.amazonaws.com)... 52.219.112.241, 16.15.0.16, 52.219.117.122, ...
Connecting to an-utd-python.s3.us-west-1.amazonaws.com (an-utd-python.s3.us-west-1.amazonaws.com)|52.219.112.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61320 (60K) [text/plain]
Saving to: ‘yelp_labelled.txt’

yelp_labelled.txt   100%[===================>]  59.88K  --.-KB/s    in 0.04s   

2025-10-18 23:08:11 (1.52 MB/s) - ‘yelp_labelled.txt’ saved [61320/61320]



In [89]:
df = spark.read.option("header","false").option("inferSchema","true").option("delimiter","\t").csv("yelp_labelled.txt").toDF("text", "label")

In [87]:
df = spark.read.option("header","false").option("inferSchema","true").option("delimiter","\t").csv("yelp_labelled.txt")

In [90]:
df.show(5)

+--------------------+-----+
|                text|label|
+--------------------+-----+
|Wow... Loved this...|    1|
|  Crust is not good.|    0|
|Not tasty and the...|    0|
|Stopped by during...|    1|
|The selection on ...|    1|
+--------------------+-----+
only showing top 5 rows



In [54]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(df)

In [56]:
type(wordsData)
wordsData.show(5)

+--------------------+-----+--------------------+
|                text|label|               words|
+--------------------+-----+--------------------+
|Wow... Loved this...|    1|[wow..., loved, t...|
|  Crust is not good.|    0|[crust, is, not, ...|
|Not tasty and the...|    0|[not, tasty, and,...|
|Stopped by during...|    1|[stopped, by, dur...|
|The selection on ...|    1|[the, selection, ...|
+--------------------+-----+--------------------+
only showing top 5 rows



In [57]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
stopWordRemoved = remover.transform(wordsData)
stopWordRemoved.show(5)

+--------------------+-----+--------------------+--------------------+
|                text|label|               words|            filtered|
+--------------------+-----+--------------------+--------------------+
|Wow... Loved this...|    1|[wow..., loved, t...|[wow..., loved, p...|
|  Crust is not good.|    0|[crust, is, not, ...|      [crust, good.]|
|Not tasty and the...|    0|[not, tasty, and,...|[tasty, texture, ...|
|Stopped by during...|    1|[stopped, by, dur...|[stopped, late, m...|
|The selection on ...|    1|[the, selection, ...|[selection, menu,...|
+--------------------+-----+--------------------+--------------------+
only showing top 5 rows



In [59]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover #Feature Transformers
from pyspark.ml.feature import HashingTF, IDF, FeatureHasher  #Feature Extractors

In [72]:
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(stopWordRemoved)

In [73]:
featurizedData.show(5, truncate=False)

+---------------------------------------------------------------------------------------+-----+-------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------+
|text                                                                                   |label|words                                                                                                  |filtered                                                                    |rawFeatures                                                                                       |
+---------------------------------------------------------------------------------------+-----+-------------------------------------------------------------------------------------------------------+-------------------------------------------------

In [76]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
rescaledData.show(5,truncate=False)

+---------------------------------------------------------------------------------------+-----+-------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                   |label|words                                                                                                  |filtered                                                                    |rawFeatures                              

In [77]:
# create a logistic regression model

train, test = rescaledData.randomSplit([0.9, 0.1], seed = 6579)
train.show(5)
test.show(5)




+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|                text|label|               words|            filtered|         rawFeatures|            features|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|!....THE OWNERS R...|    0|[!....the, owners...|[!....the, owners...|(20,[2,3,4,5,7,8,...|(20,[2,3,4,5,7,8,...|
|(It wasn't busy e...|    0|[(it, wasn't, bus...|[(it, busy, eithe...|(20,[3,4,5,7,10,1...|(20,[3,4,5,7,10,1...|
|(The bathroom is ...|    1|[(the, bathroom, ...|[(the, bathroom, ...|(20,[4,7,9,11,13,...|(20,[4,7,9,11,13,...|
|* Both the Hot & ...|    1|[*, both, the, ho...|[*, hot, &, sour,...|(20,[1,2,3,6,8,11...|(20,[1,2,3,6,8,11...|
|- Really, really ...|    1|[-, really,, real...|[-, really,, real...|(20,[7,8,12,13,16...|(20,[7,8,12,13,16...|
+--------------------+-----+--------------------+--------------------+--------------------+-----

In [78]:
lr = LogisticRegression(regParam=0.3, elasticNetParam=0.8, maxIter = 1000)
model = lr.fit(train)
result = model.transform(test)
result.show(5, truncate=False)


+-------------------------------------------+-----+-----------------------------------------------------+------------------------------------+-----------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------+----------------------------------------+----------+
|text                                       |label|words                                                |filtered                            |rawFeatures                                    |features                                                                                                                                |rawPrediction                               |probability                             |prediction|
+-------------------------------------------+-----+-----------------------------------------------------+------------------------------------+--------

In [79]:
type(model)

pyspark.ml.classification.LogisticRegressionModel

In [80]:
evaluator = MulticlassClassificationEvaluator(metricName = "accuracy")
accuracy = evaluator.evaluate(result)
print(accuracy)

0.4895833333333333


In [42]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover #Feature Transformers
from pyspark.ml.feature import HashingTF, IDF, FeatureHasher  #Feature Extractors

In [48]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

sentenceDataFrame = spark.createDataFrame([
    (0, "Hi I heard about Spark"),
    (1, "I wish Java could use case classes"),
    (2, "Logistic,regression,models,are,neat")
], ["id", "sentence"])

tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

regexTokenizer = RegexTokenizer(inputCol="sentence", outputCol="words", pattern="\\W", gaps=True)
# alternatively, pattern="\\w+", gaps(False)

countTokens = udf(lambda words: len(words), IntegerType())

tokenized = tokenizer.transform(sentenceDataFrame)
tokenized.select("sentence", "words")\
    .withColumn("tokens", countTokens(col("words"))).show(truncate=False)

regexTokenized = regexTokenizer.transform(sentenceDataFrame)
regexTokenized.select("sentence", "words") \
.withColumn("tokens", countTokens(col("words"))).show(truncate=False)

+-----------------------------------+------------------------------------------+------+
|sentence                           |words                                     |tokens|
+-----------------------------------+------------------------------------------+------+
|Hi I heard about Spark             |[hi, i, heard, about, spark]              |5     |
|I wish Java could use case classes |[i, wish, java, could, use, case, classes]|7     |
|Logistic,regression,models,are,neat|[logistic,regression,models,are,neat]     |1     |
+-----------------------------------+------------------------------------------+------+

+-----------------------------------+------------------------------------------+------+
|sentence                           |words                                     |tokens|
+-----------------------------------+------------------------------------------+------+
|Hi I heard about Spark             |[hi, i, heard, about, spark]              |5     |
|I wish Java could use case cla

In [69]:
sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)

wordsData.show(truncate=False)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)
# alternatively, CountVectorizer can also be used to get term frequency vectors

featurizedData.show(truncate=False)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

rescaledData.select("label", "features").show(truncate=False)

+-----+-----------------------------------+------------------------------------------+
|label|sentence                           |words                                     |
+-----+-----------------------------------+------------------------------------------+
|0.0  |Hi I heard about Spark             |[hi, i, heard, about, spark]              |
|0.0  |I wish Java could use case classes |[i, wish, java, could, use, case, classes]|
|1.0  |Logistic regression models are neat|[logistic, regression, models, are, neat] |
+-----+-----------------------------------+------------------------------------------+

+-----+-----------------------------------+------------------------------------------+-----------------------------------------------+
|label|sentence                           |words                                     |rawFeatures                                    |
+-----+-----------------------------------+------------------------------------------+----------------------------------

In [81]:
!wget https://an-utd-course.s3-us-west-1.amazonaws.com/car-milage.csv

--2025-10-18 23:47:52--  https://an-utd-course.s3-us-west-1.amazonaws.com/car-milage.csv
Resolving an-utd-course.s3-us-west-1.amazonaws.com (an-utd-course.s3-us-west-1.amazonaws.com)... 52.219.113.17, 16.15.0.248, 52.219.113.73, ...
Connecting to an-utd-course.s3-us-west-1.amazonaws.com (an-utd-course.s3-us-west-1.amazonaws.com)|52.219.113.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1569 (1.5K) [text/csv]
Saving to: ‘car-milage.csv.1’

car-milage.csv.1    100%[===================>]   1.53K  --.-KB/s    in 0s      

2025-10-18 23:47:53 (91.2 MB/s) - ‘car-milage.csv.1’ saved [1569/1569]



In [83]:
#Libraries
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

In [94]:
cars = spark.read.option("header","true"). option("inferSchema","true").csv("car-milage.csv.1")
cars.show(5)
type(cars)

+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+
|  mpg|displacement| hp|torque|CRatio|RARatio|CarbBarrells|NoOfSpeed|length|width|weight|automatic|
+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+
| 18.9|       350.0|165|   260|   8.0|   2.56|           4|        3| 200.3| 69.9|  3910|        1|
| 17.0|       350.0|170|   275|   8.5|   2.56|           4|        3| 199.6| 72.9|  3860|        1|
| 20.0|       250.0|105|   185|  8.25|   2.73|           1|        3| 196.7| 72.2|  3510|        1|
|18.25|       351.0|143|   255|   8.0|    3.0|           2|        3| 199.9| 74.0|  3890|        1|
|20.07|       225.0| 95|   170|   8.4|   2.76|           1|        3| 194.1| 71.8|  3365|        0|
+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+
only showing top 5 rows



pyspark.sql.dataframe.DataFrame

In [95]:
cars1 = cars.na.drop()

In [102]:
cars1.show(5)
cars1.count()
cars.show(100)
cars1.show(50)

+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+
|  mpg|displacement| hp|torque|CRatio|RARatio|CarbBarrells|NoOfSpeed|length|width|weight|automatic|
+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+
| 18.9|       350.0|165|   260|   8.0|   2.56|           4|        3| 200.3| 69.9|  3910|        1|
| 17.0|       350.0|170|   275|   8.5|   2.56|           4|        3| 199.6| 72.9|  3860|        1|
| 20.0|       250.0|105|   185|  8.25|   2.73|           1|        3| 196.7| 72.2|  3510|        1|
|18.25|       351.0|143|   255|   8.0|    3.0|           2|        3| 199.9| 74.0|  3890|        1|
|20.07|       225.0| 95|   170|   8.4|   2.76|           1|        3| 194.1| 71.8|  3365|        0|
+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+
only showing top 5 rows

+-----+------------+---+------+------+-------+------------+---------+------

In [103]:
assembler = VectorAssembler(inputCols=["displacement","hp","torque","CRatio","RARatio","CarbBarrells","NoOfSpeed","length","width","weight","automatic"], outputCol="features")

In [104]:
type(assembler)

pyspark.ml.feature.VectorAssembler

In [105]:
cars2 = assembler.transform(cars1)

In [109]:
cars1.show(5, truncate=False)
cars2.show(5, truncate=False)

+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+
|mpg  |displacement|hp |torque|CRatio|RARatio|CarbBarrells|NoOfSpeed|length|width|weight|automatic|
+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+
|18.9 |350.0       |165|260   |8.0   |2.56   |4           |3        |200.3 |69.9 |3910  |1        |
|17.0 |350.0       |170|275   |8.5   |2.56   |4           |3        |199.6 |72.9 |3860  |1        |
|20.0 |250.0       |105|185   |8.25  |2.73   |1           |3        |196.7 |72.2 |3510  |1        |
|18.25|351.0       |143|255   |8.0   |3.0    |2           |3        |199.9 |74.0 |3890  |1        |
|20.07|225.0       |95 |170   |8.4   |2.76   |1           |3        |194.1 |71.8 |3365  |0        |
+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+
only showing top 5 rows

+-----+------------+---+------+------+-------+------------+---------+------

In [110]:
train, test = cars2.randomSplit([0.9, 0.1], seed = 2018)

In [117]:
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, labelCol = "mpg")
lrModel = lr.fit(train)

In [118]:
type(lrModel)

pyspark.ml.regression.LinearRegressionModel

In [119]:
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

Coefficients: [-0.013749401450378392,-0.013731082589963806,-0.00933903639392948,1.5962108394471706,1.1476505260774805,0.0,0.0,0.0,-0.15360057948792488,-0.0010454034907914555,0.0]
Intercept: 25.895622643713683


In [120]:
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)


numIterations: 10
objectiveHistory: [0.5, 0.39919673548029494, 0.15918045576458048, 0.15656497645057618, 0.14950176866695794, 0.1492165966433807, 0.14895423542420252, 0.14857600070926696, 0.14842255552580655, 0.14831250345038585, 0.147802191486251]
+--------------------+
|           residuals|
+--------------------+
|  -4.001167938925384|
|  -3.022301149297494|
|  1.9004496655246985|
| -2.2320392968571703|
| -2.5637702549252257|
|   2.426669513313982|
|  2.5415002370962974|
| -2.3885697307479674|
|  -1.733535316147659|
| -2.0750165243103424|
|-0.45385085039134765|
| -0.7453215037513061|
|  0.7458405236613714|
|  1.6269820465492728|
|-0.26571482170841776|
|   2.120705269270001|
| -0.8312841462204261|
| -1.8693004339477959|
|   -5.34078481717615|
| -0.5809695828648138|
+--------------------+
only showing top 20 rows

RMSE: 3.004913
r2: 0.776213


In [121]:
test.show()


+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+--------------------+
|  mpg|displacement| hp|torque|CRatio|RARatio|CarbBarrells|NoOfSpeed|length|width|weight|automatic|            features|
+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+--------------------+
| 17.8|       302.0|129|   220|   8.0|    3.0|           2|        3| 199.9| 74.0|  3890|        1|[302.0,129.0,220....|
|22.12|       231.0|110|   175|   8.0|   2.56|           2|        3| 179.3| 65.4|  3020|        1|[231.0,110.0,175....|
+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+--------------------+



In [122]:
result = lrModel.transform(test)
result.show(5, truncate=False)

+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+----------------------------------------------------------+------------------+
|mpg  |displacement|hp |torque|CRatio|RARatio|CarbBarrells|NoOfSpeed|length|width|weight|automatic|features                                                  |prediction        |
+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+----------------------------------------------------------+------------------+
|17.8 |302.0       |129|220   |8.0   |3.0    |2           |3        |199.9 |74.0 |3890  |1        |[302.0,129.0,220.0,8.0,3.0,2.0,3.0,199.9,74.0,3890.0,1.0] |18.696981577454196|
|22.12|231.0       |110|175   |8.0   |2.56   |2           |3        |179.3 |65.4 |3020  |1        |[231.0,110.0,175.0,8.0,2.56,2.0,3.0,179.3,65.4,3020.0,1.0]|22.079836076477825|
+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+-----------

In [116]:
result.show(truncate=False)

+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+----------------------------------------------------------+------------------+
|mpg  |displacement|hp |torque|CRatio|RARatio|CarbBarrells|NoOfSpeed|length|width|weight|automatic|features                                                  |prediction        |
+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+----------------------------------------------------------+------------------+
|17.8 |302.0       |129|220   |8.0   |3.0    |2           |3        |199.9 |74.0 |3890  |1        |[302.0,129.0,220.0,8.0,3.0,2.0,3.0,199.9,74.0,3890.0,1.0] |18.696981577454196|
|22.12|231.0       |110|175   |8.0   |2.56   |2           |3        |179.3 |65.4 |3020  |1        |[231.0,110.0,175.0,8.0,2.56,2.0,3.0,179.3,65.4,3020.0,1.0]|22.079836076477825|
+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+-----------

In [126]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="mpg", metricName = "rmse")
accuracy = evaluator.evaluate(result)
print(accuracy)

0.6348972716293996
